# **QSwitch example usage**

In [2]:
from src.qswitch import QSwitch, channel_list_to_state, UdpConfig
from src.common import connection

# Connect and identify

**Connect via USB using VISA**:

- If only one QSwitch is connected - automatically find the address:

In [ ]:
device = connection.find_qswitch_on_usb()
qswitch = QSwitch(device)
print(qswitch.query("*idn?"))

- or specify the port number

In [ ]:
qswitch_portno= '5'  # 
device = connection.find_visa_device(address=f'ASRL{qswitch_portno}::INSTR',description="QSwitch")
qswitch = QSwitch(device)
print(qswitch.query("*idn?"))

In [ ]:
# If in doubt about the port number, list the available serial ports with connected QDACs or QSwitch's
print(connection.find_serial_devices())

# Try to match these up with the VISA port numbers
print(connection.resource_manager().list_resources())

**Connect via Ethernet (UDP):**  (firmware version > 1.3)

In [ ]:
qswitch_addr = '192.168.8.101'
device = UdpConfig(ip = qswitch_addr)
qswitch = QSwitch(device)
print(qswitch.query("*idn?"))

# Manipulate relays

Close all grounding relays and open all other relays by resetting the QSwitch (*rst)

In [4]:
qswitch.command("*rst")

Show closed relays

In [ ]:
print(qswitch.query("close:stat?"))

Use the "channel_to_state" helper to make a list of closed relays

In [ ]:
state = channel_list_to_state(qswitch.query("clos:stat?"))
print(state)

Connect and unground line 23:

In [ ]:
qswitch.command("close (@23!9)")
qswitch.command("open (@23!0)")
print(channel_list_to_state(qswitch.query("close:stat?")))

Tap off line 23 to BNC 2:

In [ ]:
qswitch.command("close (@23!2)")
print(qswitch.query("close:stat?"))

Multiple relays at once:

In [ ]:
qswitch.command("open (@1!0:8!0)")
print(qswitch.query("close:stat?"))

# Double check that specific relauys are open


Check that specific relays are open or closed

In [ ]:
print("Open:\t",qswitch.query("open? (@1!0:8!0)"))
print("Closed:\t",qswitch.query("close? (@1!0:8!0)"))

## Check for errors:

In [ ]:
print(qswitch.query("err:all?"))

Beep and blink on SCPI command errors (default off):

In [35]:
qswitch.command('syst:beep:stat on')

In [ ]:
qswitch.command('syst:beep:stat off')

## Autosave

When turning on autosave, the relay states will be restored after restart (SYSTem:RESTart).

In [50]:
qswitch.command('syst:autosave on')
qswitch.command('syst:aut off')

## End session (close connection)

The QSwitch only supports a single USB connection. So in order not to block it you can tell Python to release the connection. This is also good practice for ethernet connections.

In [10]:
qswitch.close()